# Custom Training StyleGan2-ADA

StyleGAN2-ADA only work with Tensorflow 1. Run the next cell before anything else to make sure we’re using TF1 and not TF2.

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
!nvidia-smi

Sun Nov 22 14:33:55 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Install Repo to Google Drive

Colab is a little funky with training. I’ve found the best way to do this is to install the repo directly into your Google Drive folder.

First, mount your Drive to the Colab notebook: 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Next, run this cell. If you’re already installed the repo, it will skip the installation process and change into the repo’s directory. If you haven’t installed it, it will install all the files necessary.

In [4]:
import os
if os.path.isdir("/content/drive/My Drive/colab-sg2-ada"):
    %cd "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada"
else:
    #install script
    %cd "/content/drive/My Drive/"
    !mkdir colab-sg2-ada
    %cd colab-sg2-ada
    !git clone https://github.com/dvschultz/stylegan2-ada
    %cd stylegan2-ada
    !mkdir downloads
    !mkdir datasets

/content/drive/My Drive/colab-sg2-ada/stylegan2-ada


In [ ]:
%cd "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git checkout origin/main -- train.py

/content/drive/My Drive/colab-sg2-ada/stylegan2-ada


## Convert dataset to .tfrecords

**Note: You only need to do this once per dataset. If you have already run this and are returning to conntinue training, skip these cells.**

Next we need to convert our image dataset to a format that StyleGAN2-ADA can read from. There are two options here. You can upload your dataset directly to Colab (as a zipped file), or you can upload it to Drive directly and read it from there.

In [5]:
%cd ~
%cd /content
!gdown --id 1CwkMmE55nrp4cExmOMLbklXUhnmcHuRw
%cd "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada"

/root
/content
Permission denied: https://drive.google.com/uc?id=1CwkMmE55nrp4cExmOMLbklXUhnmcHuRw
Maybe you need to change permission over 'Anyone with the link'?
/content/drive/My Drive/colab-sg2-ada/stylegan2-ada


In [7]:
#if you manually uploaded your dataset to Colab, unzip it
zip_path = "/content/drive/MyDrive/Training_data/expressions_training_dataset.zip"

!unzip {zip_path} -d /content/expressions/

Archive:  /content/drive/MyDrive/Training_data/expressions_training_dataset.zip
  inflating: /content/expressions/454622442000.jpg  
  inflating: /content/expressions/454622442846.jpg  
  inflating: /content/expressions/454622442845.jpg  
  inflating: /content/expressions/454622442844.jpg  
  inflating: /content/expressions/454622442843.jpg  
  inflating: /content/expressions/454622442842.jpg  
  inflating: /content/expressions/454622442841.jpg  
  inflating: /content/expressions/454622442840.jpg  
  inflating: /content/expressions/454622442839.jpg  
  inflating: /content/expressions/454622442838.jpg  
  inflating: /content/expressions/454622442837.jpg  
  inflating: /content/expressions/454622442836.jpg  
  inflating: /content/expressions/454622442835.jpg  
  inflating: /content/expressions/454622442834.jpg  
  inflating: /content/expressions/454622442833.jpg  
  inflating: /content/expressions/454622442832.jpg  
  inflating: /content/expressions/454622442831.jpg  
  inflating: /conte

Optional: Use only if you need to delete something

In [ ]:
#if you want to delete a directory

import shutil

shutil.rmtree('/content/mudra')

FileNotFoundError: ignored

In [ ]:
#if you want to delete a file

import os
os.remove("/content/expressions/.DS_Store") 

In [ ]:
#if you want to resize images

#!/usr/bin/python
from PIL import Image
import os, sys

path = "/content/mudra/"
dirs = os.listdir( path )

def resize():
    for item in dirs:
        if os.path.isfile(path+item):
            im = Image.open(path+item)
            f, e = os.path.splitext("/content/mudra2/" + item)
            imResize = im.resize((1024,1024), Image.ANTIALIAS)
            imResize.save(f + '.jpg', 'JPEG', quality=100)

resize()

Now that your image dataset is uploaded, we need to convert it to the `.tfrecords` format.

Depending on the resolution of your images and how many you have, this can take a while.

In [9]:
#update this to the path to your image folder
dataset_path = "/content/expressions"
#give your dataset a name
dataset_name = "expressions"

#you don't need to edit anything here
!python dataset_tool.py create_from_images ./datasets/{dataset_name} {dataset_path}

Loading images from "/content/expressions"
Creating dataset "./datasets/expressions"
Added 844 images.


## Train a custom model

We’re ready to start training! There are numerous arguments to training, what’s listed below are the most popular options. To see all the options, run the following cell.

In [ ]:
!python train.py --help

In [ ]:
#this name must EXACTLY match the dataset name you used when creating the .tfrecords file
dataset_name = "expressions"
#how often should the model generate samples and a .pkl file
snapshot_count = 2
#should the images be mirrored left to right?
mirrored = True
#should the images be mirrored top to bottom?
mirroredY = False
#metrics? 
metric_list = None
#augments
augs = "bg"

#
# this is the most important cell to update
#
# running it for the first time? set it to ffhq(+resolution)
# resuming? get the path to your latest .pkl file and use that
resume_from = "ffhq512"

#don't edit this unless you know what you're doing :)
!python train.py --outdir ./results --snap={snapshot_count} --cfg=11gb-gpu --data=./datasets/{dataset_name} --augpipe={augs} --mirror={mirrored} --mirrory={mirroredY} --metrics={metric_list} --resume={resume_from}

tcmalloc: large alloc 4294967296 bytes == 0x7a26000 @  0x7ff61e409001 0x7ff61b632765 0x7ff61b696bb0 0x7ff61b698a4f 0x7ff61b72f048 0x50a4a5 0x50cc96 0x507be4 0x508ec2 0x594a01 0x549e8f 0x5515c1 0x5a9dac 0x50a433 0x50cc96 0x507be4 0x588e5c 0x59fd0e 0x50d256 0x507be4 0x588e5c 0x59fd0e 0x50d256 0x507be4 0x588e5c 0x59fd0e 0x50d256 0x5095c8 0x50a2fd 0x50beb4 0x507be4
tcmalloc: large alloc 4294967296 bytes == 0x7ff44c2c2000 @  0x7ff61e4071e7 0x7ff61b6325e1 0x7ff61b696c78 0x7ff61b696f37 0x7ff61b72ef28 0x50a4a5 0x50cc96 0x507be4 0x509900 0x50a2fd 0x50cc96 0x507be4 0x509900 0x50a2fd 0x50cc96 0x507be4 0x509900 0x50a2fd 0x50cc96 0x5095c8 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50cc96 0x507be4
tcmalloc: large alloc 4294967296 bytes == 0x7ff34b2c0000 @  0x7ff61e4071e7 0x7ff61b6325e1 0x7ff61b696c78 0x7ff61b696f37 0x7ff5d969c0c5 0x7ff5d901f902 0x7ff5d901feb2 0x7ff5d8fd8c3e 0x50a12f 0x50beb4 0x5095c8 0x50a2fd 0x50beb4 0x507be4 0x588c8b 0x59fd0e 0x50d256 0x507b

### While it’s training...
**Once the above cell is running you should be training!**

Don’t close this tab! Colab needs to be open and running in order to continue training. Every ~15min or so a new line should get added to your output, indicated its still training. Depending on you `snapshot_count` setting you should see the results folder in your Google drive folder fill with both samples (`fakesXXXXXx.jpg`) and model weights (`network-snapshot-XXXXXX.pkl`). The samples are worth looking at while it trains but don’t get too worried about each individual sample.

If you chose a metric, you will also see scores for each snapshot. Don’t obsess over these! they are a guide, it can go up or down slightly for each snapshot. What you want to see is a gradual lowering of the score over time.

Once Colab shuts off, you can Reconnect the notebook and re-run every cell from top to bottom. Make sure you update the `resume_from` path to continue training from the latest model.